In [1]:
from pypendula import PyPendula
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
import numpy as np

In [2]:
FPS = 60
T_F = 1000

ALPHA = 2
BETA = 4096

ICS2 = np.array([ 0, np.pi / 2, 0, 0])
ICS3 = np.array([np.pi / 6, np.pi / 5, np.pi / 4, 0, 0, 0])

In [55]:
p2 = PyPendula(N=2, fps=FPS, t_f=T_F, ics=ICS2)
p2.plot_poincare_section()

Solving Numeric Problem... Done!


In [8]:
p3 = PyPendula(fps=FPS, t_f=T_F, ics=ICS3)
p3.plot_poincare_section()

# p.solve_symbolic()
# p.simulate()

Solving Numeric Problem... Done!


In [3]:
N = 8
fig, ax = plt.subplots(1, 1, layout="constrained", figsize=(19.2, 10.80))

ax.set_title('PyPendula Poincare Section')
ax.set_xlabel(r'$q$ [rad]')
ax.set_ylabel(r'$p$ [rad]/[s]')
    
color = cm.rainbow(np.linspace(0, 1, 10))
for i, c in enumerate(color):
    p = PyPendula(fps=FPS, t_f=T_F)
    p.solve_numeric()                
    ax.scatter(p.poincare_q, p.poincare_p, lw=0.5, color=c)

plt.savefig(f'./results/pypendula_n{p.N}_poincare_section.png')
plt.close()

Solving Numeric Problem... 

Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!
Solving Numeric Problem... Done!


In [ ]:
from sympy.physics.mechanics import *
from sympy import symbols, sin, cos, atan, Matrix, simplify, solve
from sympy.utilities import lambdify
from functools import partial
import numpy as np

rng = np.random.default_rng()

In [ ]:
N = 2

q = dynamicsymbols(f'q:{N}')
p = dynamicsymbols(f'p:{N}')
dq = dynamicsymbols(f'q:{N}', level=1)
dp = dynamicsymbols(f'p:{N}', level=1)
m = symbols(f'm:{N}')
l = symbols(f'l:{N}')
c, g, t = symbols('c g t')

# Compose World Frame
O = ReferenceFrame('O')
pin = Point('O*')
pin.set_vel(O, 0)

# Initial Mass
points = [pin.locatenew(f'P_0', l[0] * sin(q[0]) * O.x + l[0] * cos(q[0]) * O.y),]
points[0].set_vel(O, points[0].pos_from(pin).dt(O))
particles = [Particle(f'pP_0', points[0], m[0]),]
particles[0].potential_energy = -m[0] * g * points[0].pos_from(pin).dot(O.y)

for i in range(1, N):
    points.append(pin.locatenew(f'P_{i}', 
        points[i - 1].pos_from(pin) + l[i] * sin(q[i]) * O.x + l[i] * cos(q[i]) * O.y)
    )
    points[i].set_vel(O, points[i].pos_from(pin).dt(O))
    particles.append(Particle(f'pP_{i}', points[i], m[i]))
    particles[i].potential_energy = m[i] * g * points[i].pos_from(pin).dot(O.y)

L = Lagrangian(O, *particles)
LM = LagrangesMethod(L, q, frame=O)  # , forcelist=[(P, -c * P.vel(O))])

In [ ]:
EL = simplify(LM.form_lagranges_equations().subs([(dq[_], p[_]) for _ in range(N)]))
EL

In [ ]:
EOM = simplify(LM.eom.subs([(dq[_], p[_]) for _ in range(N)]))
EOM

In [ ]:
solved_EOM = Matrix(list(solve(EOM, *dp).values()))
solved_EOM